In [1]:
load('sidon_cryptosystem.sage')
##Function that returns the coeffiecient xth variable in the yth equation in the system
def getLinearizedCoeff(y, x, k, r, matrixList, c_matrix):
    if y % 100 == 0 and x == 0: 
        print(y)
    t1,a1,t2,a2,d,lx,ly,jx,jy = getIndices(y,x,k,r)

    if t1*r*k + a1 > t2*r*k + a2: 
      
        return 0
    c = c_matrix[a1][a2][d]
    coeff = c*(matrixList[t1][jy][ly]*matrixList[t2][jx][lx] - matrixList[t1][jy][lx]*matrixList[t2][jx][ly])

    coeff2 = 0
    if t1*r*k + a1 != t2*r*k + a2:
        
        temp = t1
        t1 = t2
        t2 = temp

        temp = a1
        a1 = a2
        a2 = temp
        
        c2 = c_matrix[a1][a2][d]
        coeff2 = c2*(matrixList[t1][jy][ly]*matrixList[t2][jx][lx] - matrixList[t1][jy][lx]*matrixList[t2][jx][ly])
           
    return coeff + coeff2

##Helper function that breaks y and x into the 9 indexes that specify an equation and variable in the system
def getIndices(y,x, k, r): 
    y = int(y)
    x = int(x)
    
    ##Get the variable, treating x like a base rk number
    rk  = r*k
    t1 = x//rk^3
    rem = x% rk^3 
    a1 = rem//rk^2
    rem = rem%rk^2
    t2 = rem//rk
    rem = rem%rk
    a2= rem 
    
    
    d = y//((k^2-k)*(k^2 - k)/4)
    rem = y%((k^2-k)*(k^2 - k)/4)
    l = rem//int(((k^2 - k)/2)) + 1
    rem = rem%((k^2 - k)/2)
    j = rem + 1 
    
    ##Get the equation. Indices must follow that lx > ly and jx > jy   
    lx  = 1
    jx = 1
    ly = 0
    jy = 0
    continuel = True
    continuej = True
    startl = k*(k-1)/2
    startj = k*(k-1)/2
    for i in range(k-1):
        if continuel == True:
            startl -= k-1 - i
            if l> startl:
                lx = k-1 - i 
                ly = l - startl - 1
                continuel = False
        if continuej == True:
            startj -= k-1 - i
            if j> startj:
                jx = k-1 - i 
                jy = j - startj - 1
                continuej = False
    return int(t1), int(a1), int(t2), int(a2), int(d), int(lx), int(ly), int(jx), int(jy)
            

In [9]:
##initialize parameters, and construct cryptosystem
def minorAttack(q,k):
    r = 2
    basefield = GF(q)
    rk = r*k
    y, F, F_r, d, c = ConstructSidon2k(q, k)
    matrixList, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)
    basefield = GF(q)

    ##Construct a new random basis for F_r 
    F_r_basis_new = Matrix(basefield, rk, lambda i,j: basefield.random_element())
    while F_r_basis_new.is_invertible() == False: 
         F_r_basis_new = Matrix(basefield, rk, lambda i,j: basefield.random_element())
    cob_matrix = F_r_basis_new.inverse()
    basiselements = []
    ##represent the multiplication table of the new basis over that basis
    for i in range(rk): 
        basiselements += [convertFromLong(F_r_basis_new[i], F, F_r)]
    c_matrix = [[0 for i in range(rk)] for j in range(rk)]
    new_mult_table = [[i*j for i in basiselements] for j in basiselements]
    for i in range(rk): 
        for j in range(rk): 
            element = convertToLong(new_mult_table[i][j])
            c_matrix[i][j] = element*cob_matrix
    ##Construct the coefficient matrix for the system. Not efficient but works for small k 
    eqn_mat = Matrix(basefield, (k^3*(k-1)^2)/2, rk^4, lambda i,j: getLinearizedCoeff(i,j, k, r, matrixList, c_matrix))
    R = PolynomialRing(GF(q),'x', 36, order = "degrevlex")
    indeterminates = R.gens()
    R.inject_variables()
    pivot_cols = eqn_mat.transpose().pivot_rows()
    eqn_mat = eqn_mat.matrix_from_rows(eqn_mat.pivot_rows())
    indices = []
    for i in range(eqn_mat.ncols()):
        if eqn_mat.transpose()[i] == zero_vector(eqn_mat.nrows()):
            continue
        indices += [i]
    eqn_mat = eqn_mat.transpose().matrix_from_rows(indices).transpose()
    monomial_order = []
    for i in indices:
        t1,a1,t2,a2,d,lx,ly,jx,jy =  getIndices(0,i,k,r)
        monomial_order += [indeterminates[t1*r*k + a1]*indeterminates[t2*r*k + a2]]
    ideallist = list(eqn_mat*vector(monomial_order).column())
    ideallist = [R(i) for i in ideallist]
    I = ideal(ideallist)
    return I

In [27]:
##Construct the coefficient matrix for the system. Not efficient but works for small k 
eqn_mat = Matrix(basefield, (k^3*(k-1)^2)/2, rk^4, lambda i,j: getLinearizedCoeff(i,j, k, r, matrixList, c_matrix))

0


In [29]:
##Construct a multivariate polynomial ring in rk ^ 2 variables 
R = PolynomialRing(GF(q),'x', 36)
indeterminates = R.gens()
R.inject_variables()

Defining x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, x35


In [30]:
eqn_mat.rank()

36

In [31]:
##remove linearly dependent equations
pivot_cols = eqn_mat.transpose().pivot_rows()
eqn_mat = eqn_mat.matrix_from_rows(eqn_mat.pivot_rows())


In [32]:
##remove the columns of 0s (where a monomial was repeated in constructing the matrix)
indices = []
for i in range(eqn_mat.ncols()):
    if eqn_mat.transpose()[i] == zero_vector(eqn_mat.nrows()):
        continue
    indices += [i]
eqn_mat = eqn_mat.transpose().matrix_from_rows(indices).transpose()

In [33]:
##Order the monomials in the system as they are ordered in the coefficient matrix
monomial_order = []
for i in indices:
    t1,a1,t2,a2,d,lx,ly,jx,jy =  getIndices(0,i,k,r)
    monomial_order += [indeterminates[t1*r*k + a1]*indeterminates[t2*r*k + a2]]

In [34]:
monomial_order

[x0^2,
 x0*x1,
 x0*x2,
 x0*x3,
 x0*x4,
 x0*x5,
 x0*x6,
 x0*x7,
 x0*x8,
 x0*x9,
 x0*x10,
 x0*x11,
 x0*x12,
 x0*x13,
 x0*x14,
 x0*x15,
 x0*x16,
 x0*x17,
 x0*x18,
 x0*x19,
 x0*x20,
 x0*x21,
 x0*x22,
 x0*x23,
 x0*x24,
 x0*x25,
 x0*x26,
 x0*x27,
 x0*x28,
 x0*x29,
 x0*x30,
 x0*x31,
 x0*x32,
 x0*x33,
 x0*x34,
 x0*x35,
 x1^2,
 x1*x2,
 x1*x3,
 x1*x4,
 x1*x5,
 x1*x6,
 x1*x7,
 x1*x8,
 x1*x9,
 x1*x10,
 x1*x11,
 x1*x12,
 x1*x13,
 x1*x14,
 x1*x15,
 x1*x16,
 x1*x17,
 x1*x18,
 x1*x19,
 x1*x20,
 x1*x21,
 x1*x22,
 x1*x23,
 x1*x24,
 x1*x25,
 x1*x26,
 x1*x27,
 x1*x28,
 x1*x29,
 x1*x30,
 x1*x31,
 x1*x32,
 x1*x33,
 x1*x34,
 x1*x35,
 x2^2,
 x2*x3,
 x2*x4,
 x2*x5,
 x2*x6,
 x2*x7,
 x2*x8,
 x2*x9,
 x2*x10,
 x2*x11,
 x2*x12,
 x2*x13,
 x2*x14,
 x2*x15,
 x2*x16,
 x2*x17,
 x2*x18,
 x2*x19,
 x2*x20,
 x2*x21,
 x2*x22,
 x2*x23,
 x2*x24,
 x2*x25,
 x2*x26,
 x2*x27,
 x2*x28,
 x2*x29,
 x2*x30,
 x2*x31,
 x2*x32,
 x2*x33,
 x2*x34,
 x2*x35,
 x3^2,
 x3*x4,
 x3*x5,
 x3*x6,
 x3*x7,
 x3*x8,
 x3*x9,
 x3*x10,
 x3*x11,
 x3*x12,
 x3

In [35]:
##Get the ideal corresponding to the system
ideallist = list(eqn_mat*vector(monomial_order).column())
ideallist = [R(i) for i in ideallist]
I = ideal(ideallist)

In [36]:
I.degree_of_semi_regularity()

ValueError: This function requires an overdefined system of polynomials.

In [ ]:
##Try to calculate the groebner basis of the ideal. This is the hard part (uses lots of memory!!)
##Need to replace with fGB library implementation 
gb = I.groebner_basis(prot = True)

In [61]:
a, v = gb.coefficient_matrix()

In [49]:
ideallist.

AttributeError: 'list' object has no attribute 'ring'

In [52]:
R(ideallist)

TypeError: Could not find a mapping of the passed element to this ring.

In [64]:
v.str()

'[  x0^2]\n[ x0*x1]\n[ x0*x2]\n[ x0*x3]\n[ x0*x4]\n[ x0*x5]\n[ x0*x6]\n[ x0*x7]\n[ x0*x8]\n[ x0*x9]\n[x0*x10]\n[x0*x11]\n[x0*x12]\n[x0*x13]\n[x0*x14]\n[x0*x15]\n[x0*x16]\n[x0*x17]\n[x0*x18]\n[x0*x19]\n[x0*x20]\n[x0*x21]\n[x0*x22]\n[x0*x23]\n[x0*x24]\n[x0*x25]\n[x0*x26]\n[x0*x27]\n[x0*x28]\n[x0*x29]\n[x0*x30]\n[x0*x31]\n[x0*x32]\n[x0*x33]\n[x0*x34]\n[x0*x35]'

(2, 1, 2, 2, 1, 5, 4, 1, 2, 2, 1, 5, 3, 6, 5, 5, 6, 2, 0, 0, 0, 0, 0, 0, 3, 6, 5, 5, 6, 2, 0, 0, 0, 0, 0, 0)

(0, 0, 0)

In [ ]:
indices

In [135]:
##experimental function 

def F_matrix_coeff(y,x,k,A, B = None): 
        ##print(y,x)
        r = 1 
        t1, a1, t2, a2, c, jx,jy, lx, ly = getIndices(y,x, k, r)
        if( lx*k^2 + ly*k > jx * k^2 + jy*k): 
            return 0
        if B == None: 

            if x < r*k:

                return A[x][jy][ly]*A[x][jx][lx] - A[x][jy][lx]*A[x][jx][ly]
            else: 

                x = (r*k+1)*(r*k)/2 - x
                ##x = x - rk
                start = r*k*(r*k-1)/2
                x1 = 1
                x2 = 0
                for i in range(r*k-1):
                    start -= r*k-1 - i
                    if x> start:
                        x1 = r*k-1 - i 
                        x2 = x - start - 1
                        break
                ##print(x1, x2)
                return A[x1][jy][ly]*A[x2][jx][lx] - A[x1][jy][lx]*A[x2][jx][ly] + A[x2][jy][ly]*A[x1][jx][lx] - A[x2][jy][lx]*A[x1][jx][ly]


        if B != None: 
            x1 = x // int(k)
            x2 = x % int(k)
            return A[x1][jy][ly]*B[x2][jx][lx] - A[x1][jy][lx]*B[x2][jx][ly] + B[x2][jy][ly]*A[x1][jx][lx] - B[x2][jy][lx]*A[x1][jx][ly]

            


In [534]:
q = 7
basefield = GF(q)
k = 7
r = 2
rk = r*k
y, F, F_r, c, d = ConstructSidon2k(q, k)
matrixList, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)
basefield = GF(q)
print(y)

[0 3 3 5 4 3 1 0 5 4 4 1 2 6]
[3 0 5 5 4 2 5 2 2 5 0 4 5 4]
[2 3 3 3 3 4 1 1 1 3 3 6 6 0]
[4 6 6 2 3 6 0 1 1 6 0 1 4 1]
[6 3 1 3 4 1 6 1 0 3 1 1 6 6]
[5 6 2 1 1 0 6 6 3 6 3 2 2 0]
[3 0 2 4 6 5 5 5 0 6 4 0 6 4]
[0 0 3 1 0 2 2 2 0 6 5 0 2 1]
[2 6 1 3 1 2 5 3 6 2 6 1 1 0]
[4 4 0 3 6 6 1 6 3 3 0 6 5 1]
[0 2 0 6 4 0 5 4 0 6 4 5 0 6]
[1 0 2 2 2 4 2 6 4 1 5 5 2 0]
[1 0 4 1 6 5 2 0 6 6 3 4 4 0]
[5 2 1 3 2 4 3 2 6 1 3 6 0 5]
6*a + 5*z^6 + 4*z^5 + 2*z^4 + 3*z^2 + 5*z + 6


In [536]:
origbasis_q = [vector(F(origbasis[i])^q) for i in range(k)]
origbasis_q = Matrix(origbasis_q)
origbasis = Matrix([vector(F(origbasis[i])) for i in range(k)])
origbasiselements = [F(origbasis[i]) for i in range(k)]
E_ = (origbasis_q*origbasis.inverse()).transpose()

In [537]:
mult_table = vector(origbasiselements).column()*vector(origbasiselements).row()

In [538]:
cob_matrix = origbasis.inverse()

vec_list = [[0 for i in range(k)] for j in range(k)]

##Generate the public key M(V,B)
for i in range(k): 
    for j in range(k):
        element = mult_table[i][j]
        long_representation = vector(element)
        vec_list[i][j] = list(long_representation*cob_matrix) 
D = [0 for i in range(k)]

##Return the public key 
for i in range(k): 
    D[i] = Matrix(basefield, k, lambda l,j: vec_list[l][j][i])

In [539]:
F = [D[i]*E_ + E_.transpose()*D[i] for i in range(k)]
G = [E_.transpose()*D[i]*E_ for i in range(k)]

In [515]:
D_D = Matrix((k*(k-1)/2)^2, k*(k+1)/2, lambda y,x: F_matrix_coeff(y,x,k,D) )
F_F = Matrix((k*(k-1)/2)^2, k*(k+1)/2, lambda y,x: F_matrix_coeff(y,x,k,F) )
G_G = Matrix((k*(k-1)/2)^2, k*(k+1)/2, lambda y,x: F_matrix_coeff(y,x,k,G) )
D_F = Matrix((k*(k-1)/2)^2, k^2, lambda y,x: F_matrix_coeff(y,x,k,D,F) )
D_G = Matrix((k*(k-1)/2)^2, k^2, lambda y,x: F_matrix_coeff(y,x,k,D,G) )
F_G = Matrix((k*(k-1)/2)^2, k^2, lambda y,x: F_matrix_coeff(y,x,k,F,G) )

[0 4 1]
[0 1 1]
[3 0 2]

[4 3 3]
[2 1 2]
[4 4 0]

In [411]:
print(D_D.rank(),F_F.rank(),G_G.rank(),D_F.rank(),D_G.rank(),F_G.rank())

15 21 15 24 15 24


In [412]:
short_stack = D_D.transpose().stack(F_F.transpose().stack(D_F.transpose()))
short_stack.rank()

54

In [389]:
full_stack = D_D.transpose().stack(G_G.transpose().stack(F_F.transpose().stack( D_F.transpose().stack(D_G.transpose().stack( F_G.transpose()))))).transpose()

In [390]:
full_stack.rank()

54

In [112]:
def fakePublicKey(y,q, F, F_r): 
    ##Generates the public key (and associated private key) using info from the given sidon space 
    p = F.characteristic()
    k = len(F.modulus().list()) - 1
    print(k)
    r = len(F_r.modulus().list()) - 1

    rk = (len(F_r.modulus().list()) - 1)*k
    basefield = GF(q)
    iterations = 0
    iterations2 = 0
    
    ##Construct bases for the sidon space as well as F_r
    sidonbasis = Matrix(basefield, k, lambda i,j: basefield.random_element()*0)
    while sidonbasis.is_invertible() == False and iterations < 500:
        element = F.random_element()
        sidonbasis = Matrix([list(vector(element^(q^i))) for i in range(k)])
        iterations += 1
   
    
    ##Get the multiplication table from the basis of the sidon space
    print(sidonbasis[0])
    v = [F(list(sidonbasis[i])) for i in range(k)]
    origbasis = sidonbasis
    ##sidonbasis = [j + j^q*y for  j in v]
    sidonbasis = [j + j^q*y for  j in v]
    
    
    mult_table = vector(sidonbasis).column()*vector(sidonbasis).row()

   

    extended_basis = [[j*y^i for j in v] for i in range(r)]
    vec_extended_basis  = []
    for j in extended_basis:
        vec_extended_basis += j
    extended_basis  = Matrix([convertToLong(i) for i in vec_extended_basis])
    assert(extended_basis.is_invertible() == True)
    print(extended_basis.nrows(), extended_basis.ncols())
    F_r_basis = extended_basis
    assert(iterations<500 and iterations2<500)
    cob_matrix = F_r_basis.inverse()
    print(cob_matrix)
    vec_list = [[0 for i in range(k)] for j in range(k)]

    ##Generate the public key M(V,B)
    for i in range(k): 
        for j in range(k):
            element = mult_table[i][j]
            long_representation = convertToLong(element)
            vec_list[i][j] = list(long_representation*cob_matrix) 
    matrixlist = [0 for i in range(rk)]
    
    ##Return the public key 
    for i in range(rk): 
        matrixlist[i] = Matrix(basefield, k, lambda l,j: vec_list[l][j][i])
    return matrixlist, sidonbasis, mult_table, F_r_basis, origbasis

In [340]:
y

4*a

In [535]:
origbasis_y = [convertToLong(F(origbasis[i])*y*y^-1) for i in range(k)] + [convertToLong(y*F(origbasis[i])) for i in range(k)] 
origbasis_y = Matrix(origbasis_y)
origbasis_y2 = Matrix([convertToLong(F(origbasis[i])*y^2) for i in range(k)])
origbasiselements = [F(origbasis[i])*y*y^-1 for i in range(k)] + [y*F(origbasis[i]) for i in range(k)] 
R_ = origbasis_y2*origbasis_y.inverse()

In [540]:
R_ = R_.transpose()
R1 = R_[0:k]
R2 = R_[k:]

In [541]:
W_ = [D[i] + sum([R1[i][j]*G[j] for j in range(k)]) for i in range(k)]

In [542]:
Q_ = [F[i] + sum([R2[i][j]*G[j] for j in range(k)]) for i in range(k)]

In [519]:
R_

[2 1 2 4 1]
[1 4 4 3 4]
[3 2 2 1 2]
[3 0 3 0 1]
[0 4 1 0 2]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

In [335]:
F(origbasis[i])

2*z^4 + 4*z^3 + z^2 + 3*z

In [543]:
W_W = Matrix((k*(k-1)/2)^2, k*(k+1)/2, lambda y,x: F_matrix_coeff(y,x,k,W_) )
Q_Q = Matrix((k*(k-1)/2)^2, k*(k+1)/2, lambda y,x: F_matrix_coeff(y,x,k,Q_) )
Q_W = Matrix((k*(k-1)/2)^2, k^2, lambda y,x: F_matrix_coeff(y,x,k,Q_,W_) )

In [544]:
print(W_W.rank(),Q_Q.rank(),Q_W.rank())
short_stack = W_W.transpose().stack(Q_Q.transpose().stack(Q_W.transpose()))
short_stack.rank()

21 28 42


77

In [447]:
W_W.ncols() + Q_Q.ncols() + Q_W.ncols()

55

In [382]:
R_

[4 3 1 2 0 3 1 0 4 3 2 2]
[1 3 1 3 2 3 3 1 4 2 3 4]
[4 1 0 0 0 0 0 1 0 0 0 1]
[1 0 0 1 3 4 1 3 4 3 2 2]
[1 4 3 0 4 2 4 3 1 2 0 2]
[1 3 4 1 4 4 4 2 3 1 3 4]

In [430]:
Q_W.ncols()

21

In [7]:
I = minorAttack(3,3)

[1 1 0 1 0 0]
[2 1 1 1 0 0]
[1 2 0 2 0 2]
[0 1 0 0 2 0]
[0 2 1 2 2 1]
[2 0 0 2 2 2]
0
Defining x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, x35


In [8]:
I

Ideal (x1^2 + x0*x2 - x1*x2 + x2^2 + x0*x3 + x1*x3 + x2*x3 - x3^2 + x0*x4 - x1*x4 - x3*x4 - x4^2 + x0*x5 + x1*x5 + x2*x5 + x4*x5 - x5^2 - x2*x6 - x3*x6 - x4*x6 - x5*x6 + x1*x7 + x2*x7 - x3*x7 + x4*x7 - x5*x7 - x0*x8 + x1*x8 + x2*x8 - x3*x8 - x5*x8 - x0*x9 - x1*x9 - x2*x9 - x3*x9 + x4*x9 - x0*x10 + x1*x10 + x3*x10 - x4*x10 - x5*x10 - x0*x11 - x1*x11 - x2*x11 - x4*x11 - x5*x11 - x13^2 - x12*x14 + x13*x14 - x14^2 - x12*x15 - x13*x15 - x14*x15 + x15^2 - x12*x16 + x13*x16 + x15*x16 + x16^2 - x12*x17 - x13*x17 - x14*x17 - x16*x17 + x17^2 + x2*x18 + x3*x18 + x4*x18 + x5*x18 + x14*x18 + x15*x18 + x16*x18 + x17*x18 - x1*x19 - x2*x19 + x3*x19 - x4*x19 + x5*x19 - x13*x19 - x14*x19 + x15*x19 - x16*x19 + x17*x19 - x19^2 + x0*x20 - x1*x20 - x2*x20 + x3*x20 + x5*x20 + x12*x20 - x13*x20 - x14*x20 + x15*x20 + x17*x20 - x18*x20 + x19*x20 - x20^2 + x0*x21 + x1*x21 + x2*x21 + x3*x21 - x4*x21 + x12*x21 + x13*x21 + x14*x21 + x15*x21 - x16*x21 - x18*x21 - x19*x21 - x20*x21 + x21^2 + x0*x22 - x1*x22 - x3*x22 